In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import random

In [ ]:
cookies = {'cookie': 'bid=AJ6_OayiiBg; __utmc=30149280; ll="118183"; dbcl2="265430210:GrHEOM+GvuI"; ck=HTry; frodotk_db="cdf121e214a966b169d365a63dfc3413"; push_noty_num=0; push_doumail_num=0; __utmv=30149280.26543; ct=y; _pk_ref.100001.8cb4=%5B%22%22%2C%22%22%2C1698579281%2C%22https%3A%2F%2Fsearch.douban.com%2F%22%5D; _pk_id.100001.8cb4=53b3d375e86fd2d7.1698579281.; ap_v=0,6.0; __utma=30149280.1185923290.1698545525.1698587290.1698594329.8; __utmb=30149280.0.10.1698594329; __utmz=30149280.1698594329.8.3.utmcsr=google|utmccn=(organic)|utmcmd=organic|utmctr=(not%20provided)'}

In [ ]:
iplist=[]
with open("verify_proxies.txt") as f:
    iplist = f.readlines()

In [ ]:
def getip():
    proxy= iplist[random.randint(0,len(iplist)-1)]
    proxy = proxy.replace("\n","")
    proxies={
        'http':'http://'+str(proxy),
        #'https':'https://'+str(proxy),
    }
    return proxies

In [ ]:
movie_data = {
    'ID': 1,
    '电影名称': '电影名称1',
    '导演': '导演1',
    '编剧': '编剧1',
    '主演': '主演1',
    '类型': '类型1',
    '制片国家/地区': '国家1',
    '语言': '语言1',
    '上映时间': '2023-01-01',
    '片长': 120,
    '又名': '又名1',
    'IMDB': 'IMDB1',
    '评分': 8.0,
    '简介': '这是电影1的简介'
}
movie_reptile_list = ['ID','电影名称','导演','编剧','主演','类型','制片国家/地区','语言','上映时间','片长','又名','IMDB','评分','简介']
movie_reptile = pd.DataFrame(columns=movie_reptile_list)

In [ ]:
movie_tag = pd.read_csv('Movie_tag.csv')

In [ ]:
# 构造随机函数
def get_ua():
    user_agents = [
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36',
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36',
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:90.0) Gecko/20100101 Firefox/90.0',
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.164 Safari/537.36',
        'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36',
        'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36',
        'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.1.1 Safari/605.1.15',
        'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36',
        'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.1.2 Safari/605.1.15',
        'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:90.0) Gecko/20100101 Firefox/90.0',
        'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.164 Safari/537.36',
        'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:90.0) Gecko/20100101 Firefox/90.0',
    ]
    user_agent = random.choice(user_agents) # 随机抽取对象
    return user_agent

In [ ]:
def reptile(ID,tag):
    url = 'https://movie.douban.com/subject/'+str(ID)
    # headers = {
    # 'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) \
    # AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36'
    # }
    ua = get_ua()
    headers = {'User-Agent': ua}
    html_content = requests.get(url = url, headers = headers, proxies = getip(),cookies = cookies)
    if html_content.status_code == 200:
        print("请求成功")
    else:
        print("请求失败，状态码：" + str(html_content.status_code))
    soup = BeautifulSoup(html_content.text, "html.parser")
    #电影名称
    movie_name = soup.find('span', property='v:itemreviewed')
    if movie_name:
        movie_name = [movie_name.get_text()]
    else:
        pass
    # 提取导演信息
    directors = []
    director_tags = soup.find_all('span', {'class': 'attrs'})
    for director_tag in director_tags:
        directors.extend([a.get_text() for a in director_tag.find_all('a', {'rel': 'v:directedBy'})])
    # 提取编剧信息
    writers = []
    writer_span = soup.find('span', class_='pl', string='编剧')
    if(writer_span):
        writer_span = writer_span.find_next('span', class_='attrs')
        for writer in writer_span:
            writers.append(writer.get_text())
    else:
        pass
    # 提取主演信息
    actors = [a.get_text() for a in soup.find_all('a', {'rel': 'v:starring'})]
    # 提取类型信息
    genres = [span.get_text() for span in soup.find_all('span', {'property': 'v:genre'})]
    if isinstance(tag, str):  # 检查是否是字符串
        tag_list = tag.split(',')
        merged_set = set(genres) | set(tag_list)
        genres = list(merged_set)
    else:
        pass
    # 制片国家
    country_element = soup.find('span', string='制片国家/地区:')
    if(country_element):
        country_element = [country_element.next_sibling.strip()]
    else:
        country_element = []
    #语言
    language =   soup.find('span', string='语言:')
    if(language):
        language = [language.next_sibling.strip()]
    else:
        language = []
    #上映时间
    release_time = []
    date_elements = soup.find_all('span', property='v:initialReleaseDate')
    for date_element in date_elements:
        release_time.append(date_element['content'])
    #片长
    movie_length = [span.get_text() for span in soup.find_all('span', property='v:runtime')]
    #别名
    other_name = []
    data = soup.find('span', string='又名:')
    if data:
        data = [data.next_sibling.strip()]
        split_names = data[0].split(' / ')
        for name in split_names:
            other_name.append(name)
    else:
        pass  
    #imdb
    imdb = soup.find('span', string='IMDb:')
    if(imdb):
        imdb = [imdb.next_sibling.strip()]
    else:
        imdb = []
    #评分
    rating_element = soup.find('strong', class_='ll rating_num')
    if(rating_element):
        rating_element = [rating_element.string]
    else:
        rating_element = []
    #简介
    Introduction = soup.find('span', property='v:summary')
    if Introduction:
        Introduction = [Introduction.get_text().replace('\u3000', '').replace('\n', '').replace(' ','')]
        # 其它操作
    else:
        Introduction = []
    movie_data['ID'] = ID
    movie_data['电影名称'] = ' '.join(movie_name)
    movie_data['导演'] = ' '.join(directors)
    movie_data['编剧'] = ' '.join(writers)
    movie_data['主演'] = ' '.join(actors)
    movie_data['类型'] = ' '.join(genres)
    movie_data['制片国家/地区'] = ' '.join(country_element)
    movie_data['语言'] = ' '.join(language)
    movie_data['上映时间'] = ' '.join(release_time)
    movie_data['片长'] = ' '.join(movie_length)
    movie_data['又名'] = ' '.join(other_name)
    movie_data['IMDB'] = ' '.join(imdb)
    movie_data['评分'] = ' '.join(rating_element)
    movie_data['简介'] = ' '.join(Introduction)


In [ ]:
for index, row in movie_tag.iterrows():
    if(index<=975):
        continue
    else:
        reptile(row['Id'],row['Tag'])
        movie_reptile  = movie_reptile._append(movie_data,ignore_index=True)
        print (movie_data)